## 株の銘柄コード一覧の取得
- 取得先
    - 日本株：[JPX](https://www.jpx.co.jp/markets/statistics-equities/misc/01.html)
    - 米国株：nasdaq

### 日本株

In [1]:
# import
import requests
import pandas as pd
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
from datetime import date
from dateutil.relativedelta import relativedelta
import os
from __future__ import annotations
from tqdm import tqdm

In [2]:
# get a list of Japanese stock codes
def stock_list_acquisition_jp(url: str,
                              section_list: list[str],
                              output_dir: str,
                              raw_name: str,
                              renamed: str) -> pd.DataFrame:
    url = url
    req = requests.get(url)
    with open(output_dir+raw_name, 'wb') as output:
        output.write(req.content)
    
    # read excel
    os.rename(output_dir+raw_name, output_dir+renamed)
    stocklist_jp_df = pd.read_excel(output_dir+renamed)
    
    # extract specific sections
    stocklist_jp_df = stocklist_jp_df.loc[stocklist_jp_df["市場・商品区分"].isin(section_list)]
    stocklist_jp_df = stocklist_jp_df.reset_index(drop=True)
    
    # rename columns
    stocklist_jp_df.rename(columns={"日付":"updated_on", "コード":"code", "銘柄名":"name", "市場・商品区分":"market_segment",
                                    "33業種コード":"33_industry_code", "33業種区分":"33_industry_segment", "17業種コード":"17_industry_code",
                                    "17業種区分":"17_industry_segment", "規模コード":"scale_code", "規模区分":"scale_segment"},
                           inplace=True)
    
    stocklist_jp_df.to_csv(output_dir+renamed[:-4]+".csv", index = False, encoding = "utf-8_sig")
    
    return stocklist_jp_df
    
os.makedirs("./output/", exist_ok = True)
url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
section_list = ["プライム（内国株式）", "スタンダード（内国株式）", "グロース（内国株式）"]
output_dir = "./output/"
raw_name = "data_j.xls"
renamed = "japanese_stock_codes_list.xls"
stocklist_jp_df = stock_list_acquisition_jp(url=url,
                                            section_list=section_list,
                                            output_dir=output_dir,
                                            raw_name=raw_name,
                                            renamed=renamed)
stocklist_jp_df

,updated_on,code,name,market_segment,33_industry_code,33_industry_segment,17_industry_code,17_industry_segment,scale_code,scale_segment
0,20230331,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20230331,1332,ニッスイ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400
2,20230331,1333,マルハニチロ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400
3,20230331,1375,雪国まいたけ,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
4,20230331,1376,カネコ種苗,スタンダード（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
...,...,...,...,...,...,...,...,...,...,...
3793,20230331,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
3794,20230331,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
3795,20230331,9995,グローセル,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
3796,20230331,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-


In [8]:
def acquire_stock_price_jp(df: pd.DataFrame,
                           n_year: int,
                           end_date: datetime,
                           output_dir: str,
                           stock_price_filename: str,
                           error_symbols_filename: str) -> pd.DataFrame:
    
    start_date = end_date - relativedelta(years = n_year)
    print(f"acquisition_range: {start_date} - {end_date}")

    stock_data_df_list = []
    error_symbols = []
    for code, name in zip(df.code, df.name):
        try:
            code_str = str(code) + ".T"
            stock_data_df = pdr.DataReader(code_str, start=start_date, end=end_date).reset_index(drop=False)
            stock_data_df["name"] = name
            stock_data_df_list.append(stock_data_df)
        except:
            print(f"{name} could not be acquired.")
            error_symbols.append(name)
    
    # dataをDataFrameに変換
    stock_price_df = pd.concat(stock_data_df_list, axis = 0, sort = False, ignore_index = True)
    # error_symbolsをDataFrameに変換
    error_symbols_df = pd.DataFrame({'error_symbols': error_symbols})
    
    # 出力
    stock_price_df.to_csv(output_dir+stock_price_filename, index = False, encoding = "utf-8_sig")
    error_symbols_df.to_csv(output_dir+error_symbols_filename, index = False, encoding = "utf-8_sig")
    
    return stock_price_df, error_symbols_df

# 取得期間
n_year = 10 # 取得年数
end_date = date.today()
stock_price_jp_df, error_symbols_jp_df = acquire_stock_price_jp(df=stocklist_jp_df,
                                                    n_year=n_year,
                                                    end_date=end_date,
                                                    output_dir=output_dir,
                                                    stock_price_filename="stock_price_jpx.csv",
                                                    error_symbols_filename="error_symbols_jpx.csv")
stock_price_jp_df

acquisition_range: 2013-04-23 - 2023-04-23
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 c

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 25935.T: No 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Date,Open,High,Low,Close,Adj Close,Volume,name
0,2013-04-23,2480.0,2780.0,2440.0,2740.0,2622.170166,1416500.0,極洋
1,2013-04-24,2760.0,2840.0,2620.0,2680.0,2564.750244,764500.0,極洋
2,2013-04-25,2640.0,2670.0,2500.0,2530.0,2421.200928,405900.0,極洋
3,2013-04-26,2520.0,2550.0,2430.0,2440.0,2335.071289,277100.0,極洋
4,2013-04-30,2490.0,2520.0,2470.0,2470.0,2363.781006,149400.0,極洋
...,...,...,...,...,...,...,...,...
8112850,2023-04-17,712.0,712.0,706.0,708.0,708.000000,44400.0,ベルーナ
8112851,2023-04-18,712.0,713.0,706.0,713.0,713.000000,101900.0,ベルーナ
8112852,2023-04-19,710.0,710.0,704.0,709.0,709.000000,94200.0,ベルーナ
8112853,2023-04-20,706.0,712.0,705.0,712.0,712.000000,70400.0,ベルーナ


In [10]:
error_symbols_jp_df

,error_symbols


In [6]:
## 検証
n_year = 10 # 取得年数
end_date = date.today()
start_date = end_date - relativedelta(years = n_year)

df=pdr.DataReader("1301.T", start=start_date, end=end_date).reset_index(drop=False)
df["name"]="極洋"
df

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,name
0,2013-04-23,2480.0,2780.0,2440.0,2740.0,2622.170166,1416500,極洋
1,2013-04-24,2760.0,2840.0,2620.0,2680.0,2564.750244,764500,極洋
2,2013-04-25,2640.0,2670.0,2500.0,2530.0,2421.200928,405900,極洋
3,2013-04-26,2520.0,2550.0,2430.0,2440.0,2335.071289,277100,極洋
4,2013-04-30,2490.0,2520.0,2470.0,2470.0,2363.781006,149400,極洋
...,...,...,...,...,...,...,...,...
2460,2023-04-17,3475.0,3480.0,3455.0,3470.0,3470.000000,14200,極洋
2461,2023-04-18,3470.0,3495.0,3470.0,3495.0,3495.000000,21500,極洋
2462,2023-04-19,3495.0,3495.0,3470.0,3480.0,3480.000000,23000,極洋
2463,2023-04-20,3490.0,3515.0,3480.0,3505.0,3505.000000,23200,極洋


### 米国株

In [11]:
# get a list of US stock codes
def stock_list_acquisition_us(output_dir: str,
                              filename: str) -> pd.DataFrame:
    
    # get a list of US stock codes
    stocklist_us_df = get_nasdaq_symbols()

    # extract specific sections
    stocklist_us_df = stocklist_us_df[stocklist_us_df["ETF"]==False]
    stocklist_us_df = stocklist_us_df.reset_index(drop=True)
    
    # output as csv
    stocklist_us_df.to_csv(output_dir+filename, encoding="utf-8_sig", index=False)

    return stocklist_us_df

output_dir = "./output/"
filename="stocklist_us.csv"
stocklist_us_df = stock_list_acquisition_us(output_dir=output_dir,
                                            filename=filename)
stocklist_us_df

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
1,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
2,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.0,False,NaN,AAC,AAC,False
3,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.0,False,NaN,AAC.U,AAC=,False
4,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.0,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
8472,True,Nasdaq Symbology Test Common Stock,Q,Q,False,100.0,True,N,NaN,ZXYZ.A,False
8473,True,NASDAQ TEST STOCK,Q,G,False,100.0,True,N,NaN,ZXZZT,False
8474,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.0,False,N,NaN,ZYME,False
8475,True,"Zynerba Pharmaceuticals, Inc. - Common Stock",Q,G,False,100.0,False,D,NaN,ZYNE,False


In [14]:
def acquire_stock_price_us(df: pd.DataFrame,
                           n_year: int,
                           end_date: datetime,
                           output_dir: str,
                           stock_price_filename: str,
                           error_symbols_filename: str) -> pd.DataFrame:
    
    start_date = end_date - relativedelta(years = n_year)
    print(f"acquisition_range: {start_date} - {end_date}")

    stock_data_df_list = []
    error_symbols = []
    for symbol, name in zip(df["NASDAQ Symbol"], df["Security Name"]):
        try:
            stock_data_df = pdr.DataReader(str(symbol), start=start_date, end=end_date).reset_index(drop=False)
            stock_data_df["name"] = name
            stock_data_df_list.append(stock_data_df)
        except:
            print(f"{symbol} could not be acquired.")
            error_symbols.append(name)
    
    # dataをDataFrameに変換
    stock_price_df = pd.concat(stock_data_df_list, axis = 0, sort = False, ignore_index = True)
    # error_symbolsをDataFrameに変換
    error_symbols_df = pd.DataFrame({'error_symbols': error_symbols})
    
    # 出力
    stock_price_df.to_csv(output_dir+stock_price_filename, index = False, encoding = "utf-8_sig")
    error_symbols_df.to_csv(output_dir+error_symbols_filename, index = False, encoding = "utf-8_sig")
    
    return stock_price_df, error_symbols_df

# 取得期間
n_year = 10 # 取得年数
end_date = date.today()
stock_price_us_df, error_symbols_us_df = acquire_stock_price_us(df=stocklist_us_df,
                                                    n_year=n_year,
                                                    end_date=end_date,
                                                    output_dir=output_dir,
                                                    stock_price_filename="stock_price_us.csv",
                                                    error_symbols_filename="error_symbols_us.csv")
stock_price_us_df

acquisition_range: 2013-04-23 - 2023-04-23
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC=: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC+: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AACT=: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACR-C: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACR-D: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACRO=: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ACRO+: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AFTR=: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AFTR+: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AKO.A: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AKO.B: No 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AMH-G: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AMH-H: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%****

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WE+: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WRAC=: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WRAC+: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WRB-E: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WRB-F: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WRB-

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- YCBD-A: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*****

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZXIET: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZXYZ.A: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZXZZT: Period 'max' is invalid, must be one of ['1d', '5d']
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,name
0,2013-04-23,30.193132,30.643778,30.135908,30.479256,27.947741,4924455.0,"Agilent Technologies, Inc. Common Stock"
1,2013-04-24,30.557940,30.786839,30.307583,30.650930,28.105156,4222799.0,"Agilent Technologies, Inc. Common Stock"
2,2013-04-25,30.793991,30.937054,30.515020,30.565092,28.026445,2963620.0,"Agilent Technologies, Inc. Common Stock"
3,2013-04-26,30.357655,30.450644,29.105865,29.542204,27.088520,7383397.0,"Agilent Technologies, Inc. Common Stock"
4,2013-04-29,29.785408,29.814020,29.363377,29.656652,27.193455,3552178.0,"Agilent Technologies, Inc. Common Stock"
...,...,...,...,...,...,...,...,...
10568581,2023-04-17,12.060000,12.190000,11.840000,11.940000,11.940000,103000.0,"Zynex, Inc. - Common Stock"
10568582,2023-04-18,11.910000,12.070000,11.570000,11.610000,11.610000,85500.0,"Zynex, Inc. - Common Stock"
10568583,2023-04-19,11.670000,12.030000,11.620000,11.980000,11.980000,107500.0,"Zynex, Inc. - Common Stock"
10568584,2023-04-20,11.850000,11.960000,11.800000,11.910000,11.910000,85900.0,"Zynex, Inc. - Common Stock"


In [15]:
error_symbols_us_df

,error_symbols


In [12]:
## 検証
n_year = 10 # 取得年数
end_date = date.today()
start_date = end_date - relativedelta(years = n_year)

df=pdr.DataReader("IBM", start=start_date, end=end_date).reset_index(drop=False)
df["name"]="IBM"
df

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,name
0,2013-04-23,180.946457,184.082214,180.315491,183.183563,122.976486,6585721,IBM
1,2013-04-24,183.709366,184.034424,182.686417,183.279160,123.040665,3863819,IBM
2,2013-04-25,184.216064,186.587006,183.862335,185.420654,124.478310,4864841,IBM
3,2013-04-26,185.621414,186.147232,184.751434,185.764816,124.709373,3650854,IBM
4,2013-04-29,186.214142,190.898666,186.089859,190.391968,127.815712,6205290,IBM
...,...,...,...,...,...,...,...,...
2513,2023-04-17,128.300003,128.720001,126.800003,127.820000,127.820000,3662700,IBM
2514,2023-04-18,128.139999,128.679993,127.349998,127.779999,127.779999,3193800,IBM
2515,2023-04-19,126.500000,126.980003,125.300003,126.320000,126.320000,7014400,IBM
2516,2023-04-20,130.149994,130.979996,125.839996,126.360001,126.360001,9749600,IBM
